In [1]:
from rfpipe import pipeline
# Updated for rfpipe version 1.3.1
from rfpipe import candidates
import rfpipe
import numpy as np 
import pylab as plt
import matplotlib
import logging
import glob
logger = logging.getLogger('rfpipe')
%matplotlib inline
from refine_utils import cd_refine

/lustre/aoc/observers/nm-10222/anaconda3/envs/fetchrf/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/lustre/aoc/observers/nm-10222/anaconda3/envs/fetchrf/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
params = {
        'axes.labelsize' : 14,
        'font.size' : 9,
        'legend.fontsize': 12,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'text.usetex': False,
        'figure.figsize': [12, 10]
        }
matplotlib.rcParams.update(params)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Set up data paths

In [3]:
datadir = '/lustre/aoc/projects/fasttransients/staging/'

SDMmininame = 'realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240'
miniSDMlist = glob.glob(datadir+SDMmininame)
workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/nbs/'
gainpath = '/home/mchammer/evladata/telcal/'

In [4]:
# datadir = '/lustre/aoc/projects/fasttransients/staging/'

# SDMmininame = 'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550'
# #'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566270772899'
# miniSDMlist = glob.glob(datadir+SDMmininame)
# workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/'
# gainpath = '/home/mchammer/evladata/telcal/'

In [5]:
gainfile = []
for sdm in miniSDMlist:
    sdmname = sdm.split('/')[-1]
    datasetId = '{0}'.format('_'.join(sdmname.split('_')[1:-1]))
    # # set the paths to the gainfile
    gainname = datasetId + '.GN'
    logging.info('Searching for the gainfile {0} in {1}'.format(gainname, gainpath))
    for path, dirs, files in os.walk(gainpath):
        for f in filter(lambda x: gainname in x, files):
            gainfile.append(os.path.join(path, gainname))
            break

2019-08-22 13:20:41,855 - root - INFO - Searching for the gainfile 18B-405_TEST_R3_A_cut.58665.73435530093.GN in /home/mchammer/evladata/telcal/


In [6]:
# gainfile = ['/home/mchammer/evladata/telcal/2019/08/19A-397.sb37006961.eb37092350.58715.036993946764.GN']

In [7]:
gainfile

['/home/mchammer/evladata/telcal/2019/07/18B-405_TEST_R3_A_cut.58665.73435530093.GN']

# Inputs

In [8]:
preffile = datadir+'realfast.yml' # Not being used currently
search_sigma = 7
refine = True
classify = True

# fine_dm_search = False

# if fine_dm_search:
#     dm = 1760
#     dm_steps = 100
#     dm_frac = 0.2

In [13]:
prefs={'saveplots': True, 'savenoise': False, 'savesols': False,
       'savecandcollection': True, 'savecanddata': True,
       'applyonlineflags': True, 'fftmode': 'cuda', 'clustercands': (4,3), 'maxdm':500}

# if fine_dm_search:
#     logging.info('Setting DM array around DM {0}, from {1} to {2} in {3} steps'.
#                  format(dm, dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))
#     prefs['dmarr'] = list(np.linspace(dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))

# Searching all miniSDMs

In [14]:
for index, sdm in enumerate(miniSDMlist):
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdm), sdmscan=1, inprefs = prefs)#preffile = preffile, name='NRAOdefaultS')
    #inprefs = prefs)
    st.prefs.workdir = workdir
    st.prefs.gainfile = gainfile[index]
    st.prefs.sigma_image1 = search_sigma
    st.prefs.savecanddata = True
    
    cc = pipeline.pipeline_scan(st)

2019-08-22 13:24:00,633 - rfpipe.metadata - INFO - Reading metadata from /lustre/aoc/projects/fasttransients/staging/realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240, scan 1
2019-08-22 13:24:01,740 - rfpipe.state - INFO - Metadata summary:
2019-08-22 13:24:01,740 - rfpipe.state - INFO - 	 Working directory and fileroot: /lustre/aoc/observers/nm-10222/rfgpu/realfast_analysis/refinement_notebooks, realfast_18B-405_TEST_R3_A_cut.58665.73435530093_1562005608240.1.1
2019-08-22 13:24:01,741 - rfpipe.state - INFO - 	 Using scan 1, source R3
2019-08-22 13:24:01,741 - rfpipe.state - INFO - 	 nants, nbl: 21, 210
2019-08-22 13:24:01,741 - rfpipe.state - INFO - 	 nchan, nspw: 896, 16
2019-08-22 13:24:01,744 - rfpipe.state - INFO - 	 Freq range: 1.012 -- 2.027
2019-08-22 13:24:01,744 - rfpipe.state - INFO - 	 Scan has 431 ints (4.3 s) and inttime 0.010 s
2019-08-22 13:24:01,744 - rfpipe.state - INFO - 	 2 polarizations: ['RR', 'LL']
2019-08-22 13:24:01,746 - rfpipe.state - INFO - 	 Id

2019-08-22 13:25:01,093 - rfpipe.search - INFO - Imaging 293 ints (0-292) in seg 0 at DM/dt 54.0/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:25:02,996 - rfpipe.search - INFO - Imaging 292 ints (0-291) in seg 0 at DM/dt 58.2/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:25:04,892 - rfpipe.search - INFO - Imaging 291 ints (0-290) in seg 0 at DM/dt 62.4/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:25:06,781 - rfpipe.search - INFO - Imaging 289 ints (0-288) in seg 0 at DM/dt 66.6/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:25:08,657 - rfpipe.search - INFO - Imaging 288 ints (0-287) in seg 0 at DM/dt 70.8/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:25:10,527 - rfpipe.search - INFO - Imaging 287 ints (0-286) in seg 0 at DM/dt 75.0/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:25:12,390 - rfpipe.search - INFO - Imaging 285 ints (0-284) in seg 0 at DM/dt 79.2/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:25:14,

2019-08-22 13:26:36,391 - rfpipe.search - INFO - Imaging 220 ints (0-219) in seg 0 at DM/dt 293.4/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:26:37,822 - rfpipe.search - INFO - Imaging 219 ints (0-218) in seg 0 at DM/dt 297.6/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:26:39,246 - rfpipe.search - INFO - Imaging 218 ints (0-217) in seg 0 at DM/dt 301.8/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:26:40,529 - rfpipe.search - INFO - Got one! SNR1 7.3 candidate at (0, 197, 72, 0, 0) and (l, m) = (-0.00169, -0.00486)
2019-08-22 13:26:40,665 - rfpipe.search - INFO - Imaging 216 ints (0-215) in seg 0 at DM/dt 306.0/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:26:42,070 - rfpipe.search - INFO - Imaging 215 ints (0-214) in seg 0 at DM/dt 310.2/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:26:43,469 - rfpipe.search - INFO - Imaging 214 ints (0-213) in seg 0 at DM/dt 314.4/1 with image 4608x5184 (uvres 42) on GPU 0
2019-08-22 13:26:44,8

2019-08-22 13:27:39,197 - rfpipe.candidates - INFO - Calculating normal probability distribution for 7 event in 893143646208.0 trials
2019-08-22 13:27:39,198 - rfpipe.reproduce - INFO - Zscore/SNR for candidates(s): [(7.114948182342712, 7.4869943), (6.805400818265577, 7.2728553), (6.889517632701082, 7.2871594), (6.961827401979237, 7.3221574), (6.77645372853965, 7.127565), (6.752267237470937, 7.064553), (6.841487427722622, 7.2729125)]
2019-08-22 13:27:45,099 - rfpipe.reproduce - INFO - Cluster 0/0 has 7 candidates and max detected SNR 7.5 at (0, 52, 21, 0, 0)
2019-08-22 13:27:45,190 - rfpipe.search - INFO - Correcting by delay/resampling 26/1 ints in single mode
2019-08-22 13:27:45,275 - rfpipe.search - INFO - Resampling data of length 309 by a factor of 1
2019-08-22 13:27:47,484 - rfpipe.search - INFO - Dedispersing up to delay shift of 26 integrations
2019-08-22 13:27:51,382 - rfpipe.reproduce - INFO - Calculated snrk of 2.5357752005215977 after detection. Adding it to CandData.
2019-

# Classify and generate refinement plots

In [29]:
cc.canddata

[CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 183, 0, 0, 0),
 CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 189, 0, 0, 0),
 CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 191, 0, 0, 0),
 CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 183, 1, 0, 0),
 CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 189, 1, 0, 0),
 CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 191, 1, 0, 0),
 CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 183, 3, 0, 0),
 CandData for scanId realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1 at loc (0, 191, 3, 0, 0),
 CandData for sc

In [31]:
# Classify the generated pickles using FETCH and generate refinement plots
sdmname = miniSDMlist[0].split('/')[-1]
for pkl in glob.glob(st.prefs.workdir+'/'+'cands_*'+sdmname.split('/')[0]+'*.pkl'):
    if classify or refine:
        logging.info('Refining and classifying pkl: {0}'.format(pkl))
        ccs = list(candidates.iter_cands(pkl, select='candcollection'))
        for cc in ccs:
            cds = cc.canddata
            if cds:
                for cd in cds:
                    if classify:
                        payload = candidates.cd_to_fetch(cd, classify=True, save_png=True, show=True, mode = 'GPU', devicenum='0')
                        logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
                    if refine:
                        logging.info('Generating Refinement plots')
                        cd_refine(cd)
            else:
                logging.info('No candidate was found in cc: {0}'.format(cc))

2019-08-20 21:54:45,162 - root - INFO - Refining and classifying pkl: /lustre/aoc/observers/nm-10222/rfgpu/Refinement/cands_realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1.pkl
2019-08-20 21:54:45,399 - root - INFO - No candidate was found in cc: CandCollection for realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550, scan 1, segment 0 with 17 candidates
2019-08-20 21:54:45,404 - rfpipe.candidates - INFO - Timewindow length is 60
2019-08-20 21:54:45,404 - rfpipe.candidates - INFO - Generating DM-time for candid realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566274015550.1.1_seg0-i183-dm0-dt0 in DM range 0.00--0.00 pc/cm3
2019-08-20 21:54:45,405 - rfpipe.candidates - INFO - Using gpu devicenum: 0


NameError: name 'device' is not defined